# TUTORIAL GO
### Presentación:    
https://docs.google.com/presentation/d/1JK39YtjPw88OTv5lA2GTS8o-TqO1JHbIfBX07EcYjQE/edit?usp=sharing

## Primeros Pasos
###  Historia:
Go (o Golang) es un lenguaje de programación moderno, creado en 2007 por Robert Griesemer, Rob Pike y Ken Thompson y lanzado en 2009. Trata de aprender de los errores de otros lenguajes de programación más antiguos, tomando lo mejor de cada uno y eliminando las limitaciones o problemas que estos nos presentan.

### Lenguaje Go: 
Go nace de la necesidad de crear un lenguaje donde:
- La programación no sea algo difícil
- El desarrollador no se sienta frustrado constantemente
- La eficiencia del programa resultante sea suficientemente buena


## Tour por el lenguaje
### Conceptos Básicos:
#### Paquetes:
Todo archivo de Go debe empezar siempre con una declaración package

In [ ]:
package main

#### Import: 
Los archivos Go casi siempre tienen declaraciones import 

In [ ]:
import "fmt"
import "math"

## EJEMPLOS
#### EJERCICIO SIMPLE

In [ ]:
package main

import (
    "fmt"
    "os"
	"time"
)

func main() {
	start := time.Now()

    f, err := os.Create("mi_archivo.txt")
    if err != nil {
        panic(err)
	}

    final := 16777215
    for i := 0; i <= final; i++ {
        _, err = f.WriteString(fmt.Sprintf("%06x\n", i))
        if err != nil {
            panic(err)
        }
    }

	elapsed := time.Since(start)
	fmt.Println(elapsed)
}



344.893278ms

#### mi_archivo.txt
000000  
000001  
000002  
000003  
000004  
000005  
000006  
000007  
000008  
000009  
00000a  
00000b  
00000c  
00000d  
00000e  
...  

#### EJERCICIO CON CONCURRENCIA

In [ ]:
package main

import (
    "fmt"
    "os"
	"time"
)

func calcNums(start, end int, f *os.File, doneCh chan struct{}){
	for i := start; i <= end; i++ {
		_, err := f.WriteString(fmt.Sprintf("%06x\n", i))
		if err != nil {
			panic(err)
		}
	}
	doneCh <- struct{}{}
}

func main() {
	start := time.Now()

    f, err := os.Create("mi_archivo_concurrente.txt")
    if err != nil {
        panic(err)
	}

	numGoRoutines := 10
	doneCh := make(chan struct{})

    final := 100000
    for i := 0; i <= final; i = i + (final/numGoRoutines) + 1 {
		paso := i + (final/numGoRoutines)
		if paso > final {
			paso = final
		}
		fmt.Printf("ejecutando %d %d\n", i, paso)
		go calcNums(i, paso, f, doneCh)
    }

	doneNum := 0
	for doneNum < numGoRoutines {
		<- doneCh
		fmt.Println("Terminé")
		doneNum++
	}
	fmt.Println("Todos han finalizado")

	elapsed := time.Since(start)
	fmt.Println(elapsed)
}



- ejecutando 0 10000
- ejecutando 10001 20001
- ejecutando 20002 30002
- ejecutando 30003 40003
- ejecutando 40004 50004
- ejecutando 50005 60005
- ejecutando 60006 70006
- ejecutando 70007 80007
- ejecutando 80008 90008
- ejecutando 90009 100000

- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé

Todos han finalizado

443.013137ms

#### mi_archivo_concurrente.txt

011177  
011178  
011179  
01117a  
01117b  
01117c  
01117d  
01117e  
01117f  
011180  
011181  
011182  
011183  
011184  
011185  
...

#### EJERCICIO CON CONCURRENCIA MEJORADO

In [ ]:
package main

import (
    "fmt"
    "os"
	"time"
	"strings"
)

func calcNums(start, end int, resultCh chan string, doneCh chan struct{}){
	var sBuilder strings.Builder
	for i := start; i <= end; i++ {
		fmt.Fprint(&sBuilder, fmt.Sprintf("%06x\n", i))
	}
	resultCh <- sBuilder.String()
	doneCh <- struct{}{}
}

func main() {
	start := time.Now()

    f, err := os.Create("mi_archivo_concurrente_mejorado.txt")
    if err != nil {
        panic(err)
	}

	outCh := make(chan string)
	doneWrite := make(chan struct{})
	//Writer
	go func() {
		for s := range outCh {
			_, err := f.WriteString(s)
			if err != nil {
				panic(err)
			}
		}
		doneWrite <- struct{}{}
	}()

	numGoRoutines := 10
	doneCh := make(chan struct{})

    final := 100000
    for i := 0; i <= final; i = i + (final/numGoRoutines) + 1 {
		paso := i + (final/numGoRoutines)
		if paso > final {
			paso = final
		}
		fmt.Printf("ejecutando %d %d\n", i, paso)
		go calcNums(i, paso, outCh, doneCh)
    }

	doneNum := 0
	for doneNum < numGoRoutines {
		<- doneCh
		fmt.Println("Terminé")
		doneNum++
	}
	close(outCh)
	<-doneWrite
	fmt.Println("Todos han finalizado")

	elapsed := time.Since(start)
	fmt.Println(elapsed)
}



- ejecutando 0 10000
- ejecutando 10001 20001
- ejecutando 20002 30002
- ejecutando 30003 40003
- ejecutando 40004 50004
- ejecutando 50005 60005
- ejecutando 60006 70006
- ejecutando 70007 80007
- ejecutando 80008 90008
- ejecutando 90009 100000

- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé
- Terminé

Todos han finalizado

104.021407ms

#### mi_archivo_concurrente_mejorado

004e22  
004e23  
004e24  
004e25  
004e26  
004e27  
004e28  
004e29  
004e2a  
004e2b  
004e2c  
004e2d  
004e2e  
004e2f  
004e30  
...